# **Safe and Personalized Medication Recommendation Project (MARS)**
## 1. MIMIC-III Data Preprocessing

#### This notebook is the first part of cleaning and organizing data from various MIMIC-III database tables for AI modeling.
### Tables Processed in This Notebook:

* **PRESCRIPTIONS**: Contains medication orders and prescriptions.
* **PATIENTS**: Identifies each unique patient (SUBJECT_ID).
* **ADMISSIONS**: Details each hospital admission (HADM_ID) for a patient.
* **LABEVENTS**: Includes all laboratory measurements, covering both inpatient and outpatient data.
* **NOTEEVENTS**: Contains clinical notes for patients.

For more information on these tables, refer to the MIMIC-III documentation from this link: [https://mimic.mit.edu/docs/iii/](http://)

## A. Data Loading and Cleaning

In [1]:
import pandas as pd

patients = pd.read_csv('/kaggle/input/mimic3/PATIENTS.csv')
admissions = pd.read_csv('/kaggle/input/mimic3/ADMISSIONS.csv')
labevents = pd.read_csv('/kaggle/input/lab-events-mimic-iii/LABEVENTS.csv')
prescriptions = pd.read_csv('/kaggle/input/mimic-iii/PRESCRIPTIONS.csv')
diagnoses_icd = pd.read_csv('/kaggle/input/diagnosis-icd-mimic-iii/DIAGNOSES_ICD.csv')
noteevents = pd.read_csv('/kaggle/input/noteevents-mimic-iii/NOTEEVENTS.csv')

/tmp/ipykernel_33/1538649813.py:6: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  prescriptions = pd.read_csv('/kaggle/input/mimic-iii/PRESCRIPTIONS.csv')
/tmp/ipykernel_33/1538649813.py:8: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  noteevents = pd.read_csv('/kaggle/input/noteevents-mimic-iii/NOTEEVENTS.csv')


In [2]:
seendf = pd.read_csv('/kaggle/input/seen-fine-tuning-data/Rich Fine Tuning Data.csv')
seendf

,SUBJECT_ID,HADM_ID,STARTDATE,drug_with_type,diagnosis,AGE,GENDER,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,ENDDATE,symptoms_cleaned,LABTESTS,drug_interactions,input_text,Interactions
0,53,155385,2170-03-20,"['clopidogrel bisulfate (MAIN) 75mg', 'metopro...","AMI inferolateral, init, Cocaine abuse-episodi...",46,male,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,2170-03-23 00:00:00,NaN,"Anion Gap 15 mEq/L, White Blood Cells 11.8 K/u...",NaN,A 46-year-old male admitted with emergency fro...,metoprolol and pantoprazole: Pantoprazole may ...
1,53,155385,2170-03-21,"['metoprolol (MAIN) 25mg', 'captopril (MAIN) 6...","AMI inferolateral, init, Cocaine abuse-episodi...",46,male,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,2170-03-22 00:00:00,NaN,"Magnesium 2.2 mg/dL, Creatine Kinase, MB Isoen...",NaN,A 46-year-old male admitted with emergency fro...,oxazepam and lorazepam: The risk or severity o...
2,53,155385,2170-03-22,"['metoprolol (MAIN) 25mg', 'lorazepam (MAIN) 1...","AMI inferolateral, init, Cocaine abuse-episodi...",46,male,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,2170-03-23 00:00:00,NaN,"Phosphate 2.8 mg/dL, Calcium, Total 8.8 mg/dL,...",NaN,A 46-year-old male admitted with emergency fro...,heparin and metoprolol: The risk or severity o...
3,53,155385,2170-03-23,"['metoprolol xl (MAIN) 50mg', 'warfarin (MAIN)...","AMI inferolateral, init, Cocaine abuse-episodi...",46,male,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,2170-03-23 00:00:00,NaN,"Phosphate 2.8 mg/dL, Calcium, Total 8.8 mg/dL,...",NaN,A 46-year-old male admitted with emergency fro...,NaN
4,1150,185937,2193-06-08,"['neoivgentamicin (MAIN) 6.5mg', 'send vial (B...","Single lb in-hosp w/o cs, Neonat jaund preterm...",1,male,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,2193-06-10 00:00:00,presents with a strong suck reflex but tires e...,"RDW 16.1 % abnormal, Neutrophils 28 %, Myelocy...",NaN,A 1-year-old male admitted with newborn from p...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,97152,184093,2103-06-26,['amiodarone (MAIN) 200mg'],"Atrial fibrillation, Pleural effusion NOS, Obs...",67,female,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,2103-06-25 00:00:00,NaN,"CK-MB Index 2.8 %, Creatine Kinase, MB Isoenzy...",NaN,A 67-year-old female admitted with elective fr...,NaN
1324,97152,184093,2103-06-27,['amiodarone (MAIN) 200mg'],"Atrial fibrillation, Pleural effusion NOS, Obs...",67,female,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,2103-06-19 00:00:00,NaN,"CK-MB Index 2.8 %, Creatine Kinase, MB Isoenzy...",NaN,A 67-year-old female admitted with elective fr...,NaN
1325,97152,184093,2103-07-01,['amiodarone (MAIN) 200mg'],"Atrial fibrillation, Pleural effusion NOS, Obs...",67,female,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,2103-06-19 00:00:00,NaN,"CK-MB Index 2.8 %, Creatine Kinase, MB Isoenzy...",NaN,A 67-year-old female admitted with elective fr...,NaN
1326,97152,184093,2103-07-04,['amiodarone (MAIN) 200mg'],"Atrial fibrillation, Pleural effusion NOS, Obs...",67,female,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,2103-06-25 00:00:00,NaN,"CK-MB Index 2.8 %, Creatine Kinase, MB Isoenzy...",NaN,A 67-year-old female admitted with elective fr...,NaN


## B. Subsampling Data to deal with fewer records due to the database size

In [3]:
import numpy as np

# Step 1: Get unique patient IDs from prescriptions
unique_patients = prescriptions['SUBJECT_ID'].unique()

# Step 2: Get IDs that were used before and should be excluded
seen_patients = seendf['SUBJECT_ID'].unique()

# Step 3: Exclude previously used IDs from the unique patients
available_patients = np.setdiff1d(unique_patients, seen_patients)

# Step 4: Randomly select a subset of patient IDs from the remaining patients
subset_size = int(0.013 * len(available_patients))
selected_patients = np.random.choice(available_patients, size=subset_size, replace=False)

# Step 5: Filter the DataFrame to include all rows corresponding to selected patients
presc_selected = prescriptions[prescriptions['SUBJECT_ID'].isin(selected_patients)]
adm_selected = admissions[admissions['SUBJECT_ID'].isin(selected_patients)]
patient_selected = patients[patients['SUBJECT_ID'].isin(selected_patients)]
noteevents_selected = noteevents[noteevents['SUBJECT_ID'].isin(selected_patients)]
labevents_selected = labevents[labevents['SUBJECT_ID'].isin(selected_patients)]
diagnoses_icd_selected = diagnoses_icd[diagnoses_icd['SUBJECT_ID'].isin(selected_patients)]


In [4]:
import numpy as np
# Step 1: Get unique patient IDs
unique_patients = prescriptions['SUBJECT_ID'].unique()

# Step 2: Randomly select a subset of patient IDs
subset_size = int(0.013 * len(unique_patients))
selected_patients = np.random.choice(unique_patients, size=subset_size, replace=False)

# Step 3: Filter the DataFrame to include all rows corresponding to selected patients
presc_selected = prescriptions[prescriptions['SUBJECT_ID'].isin(selected_patients)]

adm_selected = admissions[admissions['SUBJECT_ID'].isin(selected_patients)]

patient_selected = patients[patients['SUBJECT_ID'].isin(selected_patients)]

noteevents_selected = noteevents[noteevents['SUBJECT_ID'].isin(selected_patients)]

labevents_selected = labevents[labevents['SUBJECT_ID'].isin(selected_patients)]

diagnoses_icd_selected = diagnoses_icd[diagnoses_icd['SUBJECT_ID'].isin(selected_patients)]

## C. Data Transformation

In [5]:
# Convert date columns to datetime format
patient_selected['DOB'] = pd.to_datetime(patient_selected['DOB'])
adm_selected['ADMITTIME'] = pd.to_datetime(adm_selected['ADMITTIME'])
labevents_selected['CHARTTIME'] = pd.to_datetime(labevents_selected['CHARTTIME'])

/tmp/ipykernel_33/2506130628.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_selected['DOB'] = pd.to_datetime(patient_selected['DOB'])
/tmp/ipykernel_33/2506130628.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adm_selected['ADMITTIME'] = pd.to_datetime(adm_selected['ADMITTIME'])
/tmp/ipykernel_33/2506130628.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [6]:
# Calculate age at the time of each admission
adm_selected = adm_selected.merge(patient_selected[['SUBJECT_ID', 'DOB', 'GENDER']], on='SUBJECT_ID')
adm_selected['AGE'] = (adm_selected['ADMITTIME'].dt.year - adm_selected['DOB'].dt.year)

## D. Grouping Data

In [7]:
# Merge admissions with diagnoses
combined_data = adm_selected.merge(diagnoses_icd_selected[['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']], on=['SUBJECT_ID', 'HADM_ID'], how='left')
combined_data

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,DOB,GENDER,AGE,ICD9_CODE
0,383,302,191028,2133-03-29 02:33:00,2133-04-01 12:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,...,WHITE,2133-03-29 01:13:00,2133-03-29 03:41:00,ACUTE MYOCARDIAL INFARCTION,0,1,2074-03-11,M,59,41011
1,383,302,191028,2133-03-29 02:33:00,2133-04-01 12:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,...,WHITE,2133-03-29 01:13:00,2133-03-29 03:41:00,ACUTE MYOCARDIAL INFARCTION,0,1,2074-03-11,M,59,42821
2,383,302,191028,2133-03-29 02:33:00,2133-04-01 12:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,...,WHITE,2133-03-29 01:13:00,2133-03-29 03:41:00,ACUTE MYOCARDIAL INFARCTION,0,1,2074-03-11,M,59,41401
3,383,302,191028,2133-03-29 02:33:00,2133-04-01 12:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,...,WHITE,2133-03-29 01:13:00,2133-03-29 03:41:00,ACUTE MYOCARDIAL INFARCTION,0,1,2074-03-11,M,59,4280
4,383,302,191028,2133-03-29 02:33:00,2133-04-01 12:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,...,WHITE,2133-03-29 01:13:00,2133-03-29 03:41:00,ACUTE MYOCARDIAL INFARCTION,0,1,2074-03-11,M,59,4240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8293,57144,94301,127982,2172-12-28 07:15:00,2172-12-29 14:30:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,WHITE,NaN,NaN,ATRIAL FIBRILLATION\PHARMACOLOGICAL / NON PHAR...,0,1,2112-02-29,M,60,V4582
8294,58595,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,...,WHITE,2151-03-05 17:23:00,2151-03-05 21:06:00,SAH,1,1,2067-09-21,F,84,430
8295,58595,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,...,WHITE,2151-03-05 17:23:00,2151-03-05 21:06:00,SAH,1,1,2067-09-21,F,84,4019
8296,58595,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,...,WHITE,2151-03-05 17:23:00,2151-03-05 21:06:00,SAH,1,1,2067-09-21,F,84,41401


In [8]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8298 entries, 0 to 8297
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ROW_ID                8298 non-null   int64         
 1   SUBJECT_ID            8298 non-null   int64         
 2   HADM_ID               8298 non-null   int64         
 3   ADMITTIME             8298 non-null   datetime64[ns]
 4   DISCHTIME             8298 non-null   object        
 5   DEATHTIME             1210 non-null   object        
 6   ADMISSION_TYPE        8298 non-null   object        
 7   ADMISSION_LOCATION    8298 non-null   object        
 8   DISCHARGE_LOCATION    8298 non-null   object        
 9   INSURANCE             8298 non-null   object        
 10  LANGUAGE              6111 non-null   object        
 11  RELIGION              8298 non-null   object        
 12  MARITAL_STATUS        7640 non-null   object        
 13  ETHNICITY         

In [9]:
# Merge combined data with prescriptions
# Ensuring that prescriptions have been filtered for relevant columns if there are many unnecessary ones
presc_selected = presc_selected.merge(combined_data[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION', 'DIAGNOSIS', 'GENDER', 'AGE', 'ICD9_CODE']], on=['SUBJECT_ID', 'HADM_ID'], how='left')
presc_selected.drop_duplicates(inplace=True)
presc_selected

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,...,FORM_UNIT_DISP,ROUTE,ADMITTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,DIAGNOSIS,GENDER,AGE,ICD9_CODE
0,2047927,171,153112,NaN,2197-07-15 00:00:00,2197-07-16 00:00:00,MAIN,Alteplase,NaN,NaN,...,mg,IV DRIP,2197-07-15 14:52:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,SVC SYNDROME,M,62,99674
1,2047927,171,153112,NaN,2197-07-15 00:00:00,2197-07-16 00:00:00,MAIN,Alteplase,NaN,NaN,...,mg,IV DRIP,2197-07-15 14:52:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,SVC SYNDROME,M,62,4538
2,2047927,171,153112,NaN,2197-07-15 00:00:00,2197-07-16 00:00:00,MAIN,Alteplase,NaN,NaN,...,mg,IV DRIP,2197-07-15 14:52:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,SVC SYNDROME,M,62,V4281
3,2047927,171,153112,NaN,2197-07-15 00:00:00,2197-07-16 00:00:00,MAIN,Alteplase,NaN,NaN,...,mg,IV DRIP,2197-07-15 14:52:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,SVC SYNDROME,M,62,5849
4,2047927,171,153112,NaN,2197-07-15 00:00:00,2197-07-16 00:00:00,MAIN,Alteplase,NaN,NaN,...,mg,IV DRIP,2197-07-15 14:52:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,SVC SYNDROME,M,62,20301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846733,541837,99411,108851,NaN,2134-04-10 00:00:00,2134-04-11 00:00:00,MAIN,Polyethylene Glycol,Polyethylene Glycol,Polyethylene Glycol,...,PKT,PO/NG,2134-03-29 20:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,CONGESTIVE HEART FAILURE,F,88,57420
846734,541837,99411,108851,NaN,2134-04-10 00:00:00,2134-04-11 00:00:00,MAIN,Polyethylene Glycol,Polyethylene Glycol,Polyethylene Glycol,...,PKT,PO/NG,2134-03-29 20:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,CONGESTIVE HEART FAILURE,F,88,7905
846735,541837,99411,108851,NaN,2134-04-10 00:00:00,2134-04-11 00:00:00,MAIN,Polyethylene Glycol,Polyethylene Glycol,Polyethylene Glycol,...,PKT,PO/NG,2134-03-29 20:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,CONGESTIVE HEART FAILURE,F,88,2449
846736,541837,99411,108851,NaN,2134-04-10 00:00:00,2134-04-11 00:00:00,MAIN,Polyethylene Glycol,Polyethylene Glycol,Polyethylene Glycol,...,PKT,PO/NG,2134-03-29 20:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,CONGESTIVE HEART FAILURE,F,88,78052


In [10]:
presc_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846738 entries, 0 to 846737
Data columns (total 27 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   ROW_ID              846738 non-null  int64         
 1   SUBJECT_ID          846738 non-null  int64         
 2   HADM_ID             846738 non-null  int64         
 3   ICUSTAY_ID          567059 non-null  float64       
 4   STARTDATE           846394 non-null  object        
 5   ENDDATE             845922 non-null  object        
 6   DRUG_TYPE           846738 non-null  object        
 7   DRUG                846738 non-null  object        
 8   DRUG_NAME_POE       488772 non-null  object        
 9   DRUG_NAME_GENERIC   489080 non-null  object        
 10  FORMULARY_DRUG_CD   846391 non-null  object        
 11  GSN                 739315 non-null  object        
 12  NDC                 846101 non-null  float64       
 13  PROD_STRENGTH       846439 no

### Merging lab events with the combined dataset

In [11]:
labevents_selected

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
99869,107021,171,153112.0,51277,2197-07-20 03:16:00,13.1,13.10,%,NaN
99870,107022,171,153112.0,51279,2197-07-20 03:16:00,2.81,2.81,m/uL,abnormal
99871,107023,171,153112.0,51301,2197-07-20 03:16:00,5.3,5.30,K/uL,NaN
99872,107024,171,153112.0,51463,2197-07-20 17:30:00,RARE,NaN,NaN,NaN
99873,107025,171,153112.0,51464,2197-07-20 17:30:00,NEG,NaN,EU/dL,NaN
...,...,...,...,...,...,...,...,...,...
27819724,27732267,98802,101071.0,50820,2151-03-05 18:12:00,7.47,7.47,units,abnormal
27819725,27732268,98802,101071.0,50821,2151-03-05 18:12:00,227,227.00,mm Hg,abnormal
27819726,27732269,98802,101071.0,50823,2151-03-05 18:12:00,79,79.00,NaN,NaN
27819727,27732270,98802,101071.0,50825,2151-03-05 18:12:00,35.7,35.70,NaN,NaN


### Extracting lab results for the selected patients (method 1)


#### This method consumes less time but returns less lab results for a single patient visit ( it doesn't return all lab results)

In [10]:
import pandas as pd
from tqdm import tqdm

# Reduce the DataFrame size by selecting only necessary columns
adm_reduced = adm_selected[['SUBJECT_ID', 'ADMITTIME', 'HADM_ID']]
labevents_reduced = labevents_selected[['SUBJECT_ID','HADM_ID','CHARTTIME', 'ITEMID','VALUE', 'VALUEUOM', 'FLAG']]
# Assuming DataFrame 'labevents_reduced' is pre-sorted by 'CHARTTIME'
labevents_reduced['CHARTTIME'] = pd.to_datetime(labevents_reduced['CHARTTIME'])
presc_selected['STARTDATE'] = pd.to_datetime(presc_selected['STARTDATE'])

# Using a copy to ensure that we don't modify the original DataFrame unintentionally
labevents_reduced = labevents_reduced.copy()

results = []

# Use tqdm for the progress bar
for index, presc_row in tqdm(presc_selected.iterrows(), total=presc_selected.shape[0]):
    subid = presc_row['SUBJECT_ID']
    hadm_id = presc_row['HADM_ID']
    startdate = presc_row['STARTDATE']
    
    # Filter labevents for the same subject and hospital admission
    filtered_labevents = labevents_reduced[(labevents_reduced['SUBJECT_ID'] == subid) & (labevents_reduced['HADM_ID'] == hadm_id)].copy()
    
    if not filtered_labevents.empty:
        # Calculate the time difference safely
        filtered_labevents['TIME_DIFF'] = (filtered_labevents['CHARTTIME'] - startdate).abs()
        # Check if all entries are NaN
        if filtered_labevents['TIME_DIFF'].notna().any():
            closest_labevent = filtered_labevents.loc[filtered_labevents['TIME_DIFF'].idxmin()]
            
            # Append found data to results
            results.append({
                'SUBJECT_ID': subid,
                'HADM_ID': hadm_id,
                'STARTDATE' : presc_row['STARTDATE'],
                'ENDDATE' : presc_row['ENDDATE'],
                'ADMISSION_TYPE' : presc_row['ADMISSION_TYPE'],
                'ADMISSION_LOCATION' : presc_row['ADMISSION_LOCATION'],
                'DISCHARGE_LOCATION' : presc_row['DISCHARGE_LOCATION'],
                'DIAGNOSIS' : presc_row['DIAGNOSIS'],
                'ICD9_CODE' : presc_row['ICD9_CODE'],
                'GENDER' : presc_row['GENDER'],
                'AGE' : presc_row['AGE'],
                'DRUG': presc_row['DRUG'],
                'DRUG_TYPE': presc_row['DRUG_TYPE'],
                'PROD_STRENGTH': presc_row['PROD_STRENGTH'],
                'DOSE_VAL_RX': presc_row['DOSE_VAL_RX'],
                'DOSE_UNIT_RX': presc_row['DOSE_UNIT_RX'],
                'FORM_VAL_DISP': presc_row['FORM_VAL_DISP'],
                'FORM_UNIT_DISP': presc_row['FORM_UNIT_DISP'],
                'ROUTE': presc_row['ROUTE'],
                # Add other fields as necessary from presc_row and closest_labevent
                'LAB_CHARTTIME': closest_labevent['CHARTTIME'],
                'LAB_ITEMID': closest_labevent['ITEMID'],
                'LAB_VALUE': closest_labevent['VALUE'],
                'LAB_VALUEUOM': closest_labevent['VALUEUOM'],
                'LAB_FLAG': closest_labevent['FLAG']
            })
        else:
            # Handle cases where no valid lab event is found
            results.append({
                'SUBJECT_ID': subid,
                'HADM_ID': hadm_id,
                'STARTDATE' : presc_row['STARTDATE'],
                'ENDDATE' : presc_row['ENDDATE'],
                'ADMISSION_TYPE' : presc_row['ADMISSION_TYPE'],
                'ADMISSION_LOCATION' : presc_row['ADMISSION_LOCATION'],
                'DISCHARGE_LOCATION' : presc_row['DISCHARGE_LOCATION'],
                'DIAGNOSIS' : presc_row['DIAGNOSIS'],
                'ICD9_CODE' : presc_row['ICD9_CODE'],
                'GENDER' : presc_row['GENDER'],
                'AGE' : presc_row['AGE'],
                'DRUG': presc_row['DRUG'],
                'DRUG_TYPE': presc_row['DRUG_TYPE'],
                'PROD_STRENGTH': presc_row['PROD_STRENGTH'],
                'DOSE_VAL_RX': presc_row['DOSE_VAL_RX'],
                'DOSE_UNIT_RX': presc_row['DOSE_UNIT_RX'],
                'FORM_VAL_DISP': presc_row['FORM_VAL_DISP'],
                'FORM_UNIT_DISP': presc_row['FORM_UNIT_DISP'],
                'ROUTE': presc_row['ROUTE'],
                # Set placeholders or None for missing lab event data
                'LAB_CHARTTIME': None,
                'LAB_ITEMID': None,
                'LAB_VALUE': None,
                'LAB_VALUEUOM': None,
                'LAB_FLAG': None
            })
    else:
        # Handle cases where no lab events match the filtering criteria
        results.append({
            'SUBJECT_ID': subid,
            'HADM_ID': hadm_id,
            'STARTDATE' : presc_row['STARTDATE'],
            'ENDDATE' : presc_row['ENDDATE'],
            'ADMISSION_TYPE' : presc_row['ADMISSION_TYPE'],
            'ADMISSION_LOCATION' : presc_row['ADMISSION_LOCATION'],
            'DISCHARGE_LOCATION' : presc_row['DISCHARGE_LOCATION'],
            'DIAGNOSIS' : presc_row['DIAGNOSIS'],
            'ICD9_CODE' : presc_row['ICD9_CODE'],
            'GENDER' : presc_row['GENDER'],
            'AGE' : presc_row['AGE'],
            'DRUG': presc_row['DRUG'],
            'DRUG_TYPE': presc_row['DRUG_TYPE'],
            'PROD_STRENGTH': presc_row['PROD_STRENGTH'],
            'DOSE_VAL_RX': presc_row['DOSE_VAL_RX'],
            'DOSE_UNIT_RX': presc_row['DOSE_UNIT_RX'],
            'FORM_VAL_DISP': presc_row['FORM_VAL_DISP'],
            'FORM_UNIT_DISP': presc_row['FORM_UNIT_DISP'],
            'ROUTE': presc_row['ROUTE'],
            # Set placeholders or None for missing lab event data
            'LAB_CHARTTIME': None,
            'LAB_ITEMID': None,
            'LAB_VALUE': None,
            'LAB_VALUEUOM': None,
            'LAB_FLAG': None
        })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Display some of the results
results_df


/tmp/ipykernel_33/1366995089.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labevents_reduced['CHARTTIME'] = pd.to_datetime(labevents_reduced['CHARTTIME'])
100%|██████████| 794689/794689 [57:35<00:00, 229.96it/s]  


,SUBJECT_ID,HADM_ID,STARTDATE,ENDDATE,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,DIAGNOSIS,ICD9_CODE,GENDER,...,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE,LAB_CHARTTIME,LAB_ITEMID,LAB_VALUE,LAB_VALUEUOM,LAB_FLAG
0,580,194220,2137-03-16,2137-03-18 00:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,UPPER GI BLEED,56212,F,...,1,TAB,1,TAB,PO,2137-03-16 04:25:00,50868.0,8,mEq/L,NaN
1,580,194220,2137-03-16,2137-03-18 00:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,UPPER GI BLEED,5990,F,...,1,TAB,1,TAB,PO,2137-03-16 04:25:00,50868.0,8,mEq/L,NaN
2,580,194220,2137-03-16,2137-03-18 00:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,UPPER GI BLEED,49320,F,...,1,TAB,1,TAB,PO,2137-03-16 04:25:00,50868.0,8,mEq/L,NaN
3,580,194220,2137-03-16,2137-03-18 00:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,UPPER GI BLEED,2762,F,...,1,TAB,1,TAB,PO,2137-03-16 04:25:00,50868.0,8,mEq/L,NaN
4,580,194220,2137-03-16,2137-03-18 00:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,UPPER GI BLEED,2851,F,...,1,TAB,1,TAB,PO,2137-03-16 04:25:00,50868.0,8,mEq/L,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794684,99946,157197,2161-07-20,2161-07-21 00:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,PANCREATITIS,43882,M,...,20,mEq,10,mL,IV,2161-07-20 05:55:00,50868.0,11,mEq/L,NaN
794685,99946,157197,2161-07-20,2161-07-21 00:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,PANCREATITIS,79902,M,...,20,mEq,10,mL,IV,2161-07-20 05:55:00,50868.0,11,mEq/L,NaN
794686,99946,157197,2161-07-20,2161-07-21 00:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,PANCREATITIS,43813,M,...,20,mEq,10,mL,IV,2161-07-20 05:55:00,50868.0,11,mEq/L,NaN
794687,99946,157197,2161-07-20,2161-07-21 00:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,PANCREATITIS,43853,M,...,20,mEq,10,mL,IV,2161-07-20 05:55:00,50868.0,11,mEq/L,NaN


### Extracting lab results for the selected patients (method 2)

#### This method consumes more time but efficient with returning mosst lab results for a single patient visit 

In [ ]:
import pandas as pd
from tqdm import tqdm

# Ensure necessary columns are in reduced DataFrames for processing
adm_reduced = adm_selected[['SUBJECT_ID', 'ADMITTIME', 'HADM_ID']]
labevents_reduced = labevents_selected[['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'ITEMID', 'VALUE', 'VALUEUOM', 'FLAG']]

# Filter for abnormal lab results only
#labevents_reduced = labevents_reduced[labevents_reduced['FLAG'] == 'abnormal']

# Convert date columns to datetime format
labevents_reduced['CHARTTIME'] = pd.to_datetime(labevents_reduced['CHARTTIME'])
presc_selected['STARTDATE'] = pd.to_datetime(presc_selected['STARTDATE'])

# Create an empty list to store results
results = []

# Track used lab events to avoid reusing
used_lab_ids = set()

# Loop over each prescription row to find a matching unique lab test
for index, presc_row in tqdm(presc_selected.iterrows(), total=presc_selected.shape[0]):
    subid = presc_row['SUBJECT_ID']
    hadm_id = presc_row['HADM_ID']
    startdate = presc_row['STARTDATE']
    
    # Filter abnormal lab events for the same subject and admission
    filtered_labevents = labevents_reduced[(labevents_reduced['SUBJECT_ID'] == subid) & 
                                           (labevents_reduced['HADM_ID'] == hadm_id)].copy()
    
    # Calculate the time difference between prescription start date and each lab test time
    filtered_labevents['TIME_DIFF'] = (filtered_labevents['CHARTTIME'] - startdate).abs()
    
    # Sort by time difference to get the closest first and reset index to maintain unique row IDs
    filtered_labevents = filtered_labevents.sort_values(by='TIME_DIFF').reset_index(drop=True)
    
    lab_match_found = False
    for lab_index, lab_row in filtered_labevents.iterrows():
        # Create a unique identifier for the lab test (using relevant columns)
        lab_id = (lab_row['ITEMID'], lab_row['CHARTTIME'], lab_row['VALUE'], lab_row['VALUEUOM'])
        
        # Check if this lab test has already been used
        if lab_id not in used_lab_ids:
            # Mark this lab test as used and set lab_match_found to True
            used_lab_ids.add(lab_id)
            lab_match_found = True
            
            # Append the result with all details of the matched lab test
            results.append({
                'SUBJECT_ID': subid,
                'HADM_ID': hadm_id,
                'STARTDATE': presc_row['STARTDATE'],
                'ENDDATE': presc_row['ENDDATE'],
                'ADMISSION_TYPE': presc_row['ADMISSION_TYPE'],
                'ADMISSION_LOCATION': presc_row['ADMISSION_LOCATION'],
                'DISCHARGE_LOCATION': presc_row['DISCHARGE_LOCATION'],
                'DIAGNOSIS': presc_row['DIAGNOSIS'],
                'ICD9_CODE': presc_row['ICD9_CODE'],
                'GENDER': presc_row['GENDER'],
                'AGE': presc_row['AGE'],
                'DRUG': presc_row['DRUG'],
                'DRUG_TYPE': presc_row['DRUG_TYPE'],
                'PROD_STRENGTH': presc_row['PROD_STRENGTH'],
                'DOSE_VAL_RX': presc_row['DOSE_VAL_RX'],
                'DOSE_UNIT_RX': presc_row['DOSE_UNIT_RX'],
                'FORM_VAL_DISP': presc_row['FORM_VAL_DISP'],
                'FORM_UNIT_DISP': presc_row['FORM_UNIT_DISP'],
                'ROUTE': presc_row['ROUTE'],
                # Add details of the matched lab event
                'LAB_CHARTTIME': lab_row['CHARTTIME'],
                'LAB_ITEMID': lab_row['ITEMID'],
                'LAB_VALUE': lab_row['VALUE'],
                'LAB_VALUEUOM': lab_row['VALUEUOM'],
                'LAB_FLAG': lab_row['FLAG']
            })
            break  # Break the loop once a unique lab test is assigned to this prescription

    # If no unused lab test found, append the prescription row with None for lab details
    if not lab_match_found:
        results.append({
            'SUBJECT_ID': subid,
            'HADM_ID': hadm_id,
            'STARTDATE': presc_row['STARTDATE'],
            'ENDDATE': presc_row['ENDDATE'],
            'ADMISSION_TYPE': presc_row['ADMISSION_TYPE'],
            'ADMISSION_LOCATION': presc_row['ADMISSION_LOCATION'],
            'DISCHARGE_LOCATION': presc_row['DISCHARGE_LOCATION'],
            'DIAGNOSIS': presc_row['DIAGNOSIS'],
            'ICD9_CODE': presc_row['ICD9_CODE'],
            'GENDER': presc_row['GENDER'],
            'AGE': presc_row['AGE'],
            'DRUG': presc_row['DRUG'],
            'DRUG_TYPE': presc_row['DRUG_TYPE'],
            'PROD_STRENGTH': presc_row['PROD_STRENGTH'],
            'DOSE_VAL_RX': presc_row['DOSE_VAL_RX'],
            'DOSE_UNIT_RX': presc_row['DOSE_UNIT_RX'],
            'FORM_VAL_DISP': presc_row['FORM_VAL_DISP'],
            'FORM_UNIT_DISP': presc_row['FORM_UNIT_DISP'],
            'ROUTE': presc_row['ROUTE'],
            # Placeholder for missing lab data
            'LAB_CHARTTIME': None,
            'LAB_ITEMID': None,
            'LAB_VALUE': None,
            'LAB_VALUEUOM': None,
            'LAB_FLAG': None
        })

# Convert the list of dictionaries into a DataFrame
results_df = pd.DataFrame(results)

# Display some of the results
results_df


  7%|▋         | 55332/846738 [15:39<13:35:20, 16.18it/s]

In [ ]:
# Saving the Dataset
results_df.to_csv(r"Prescriptions with Labtest 14-11.csv")

In [15]:
results_df['SUBJECT_ID'].nunique()

510

In [14]:
# Get the unique patient IDs
unique_patients = results_df['SUBJECT_ID'].unique()

# Save the unique IDs to a text file
with open('unique patients 13-9.txt', 'w') as f:
    for patient_id in unique_patients:
        f.write(f"{patient_id}\n")


In [ ]:
#df.to_csv(r"Reduced Mimic noisy 13-9.csv")
adm_selected.to_csv(r"Reduced admission 13-9.csv")
patient_selected.to_csv(r"Reduced Patients 13-9.csv")

## New session for grouping the last dataset with note events table
#### please restart the kernel and upload the last combined dataset (data with prescripions nad lab events)

In [1]:
import pandas as pd
import numpy as np

noteevents = pd.read_csv('/kaggle/input/noteevents-mimic-iii/NOTEEVENTS.csv')
results_df = pd.read_csv('/kaggle/input/mimic-pre-final-27-11/MIMIC PRE FINAL 27-11.csv')

# Step 1: Get unique patient IDs
unique_patients = results_df['SUBJECT_ID'].unique()

noteevents_selected = noteevents[noteevents['SUBJECT_ID'].isin(unique_patients)]

/tmp/ipykernel_33/2973243832.py:4: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  noteevents = pd.read_csv('/kaggle/input/noteevents-mimic-iii/NOTEEVENTS.csv')


In [3]:
#results_df = results_df.drop(['Unnamed: 0'], axis=1)
results_df

,SUBJECT_ID,HADM_ID,STARTDATE,ENDDATE,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,DIAGNOSIS,ICD9_CODE,GENDER,...,LAB_CHARTTIME,LAB_ITEMID,LAB_VALUE,LAB_VALUEUOM,LAB_FLAG,nearest_chartdate,date_diff,CHARTDATE,extracted_symptoms,LABTESTS
0,1291,141087,2100-07-15,2100-07-23 00:00:00,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,NEWBORN,V053,F,...,NaN,NaN,NaN,NaN,NaN,2106-09-16,2254 days,NaN,NaN,"Bilirubin, Total 12.3 mg/dL, Bilirubin, Direct..."
1,1291,141087,2100-07-15,2100-07-23 00:00:00,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,NEWBORN,V290,F,...,NaN,NaN,NaN,NaN,NaN,2106-09-16,2254 days,NaN,NaN,"Bilirubin, Total 12.3 mg/dL, Bilirubin, Direct..."
2,1291,141087,2100-07-15,2100-07-23 00:00:00,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,NEWBORN,7630,F,...,NaN,NaN,NaN,NaN,NaN,2106-09-16,2254 days,NaN,NaN,"Bilirubin, Total 12.3 mg/dL, Bilirubin, Direct..."
3,1291,141087,2100-07-15,2100-07-23 00:00:00,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,NEWBORN,76527,F,...,2100-07-04 09:52:00,51277.0,16.2,%,abnormal,2106-09-16,2254 days,NaN,NaN,"Bilirubin, Total 12.3 mg/dL, Bilirubin, Direct..."
4,1291,141087,2100-07-15,2100-07-23 00:00:00,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,NEWBORN,7793,F,...,2100-07-04 09:52:00,51256.0,42,%,abnormal,2106-09-16,2254 days,NaN,NaN,"Bilirubin, Total 12.3 mg/dL, Bilirubin, Direct..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120469,29347,146160,2198-07-07,2198-07-07 00:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,MYOCARDIAL INFARCTION/CATH,41011,F,...,2198-07-03 05:12:00,51237.0,2.9,NaN,abnormal,2142-07-07,20454 days,NaN,NaN,"Urea Nitrogen, Urine 78 mg/dL, Creatinine, Uri..."
120470,29347,146160,2198-07-07,2198-07-07 00:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,MYOCARDIAL INFARCTION/CATH,V4364,F,...,2198-07-02 02:18:00,51274.0,33.0,sec,abnormal,2142-07-07,20454 days,NaN,NaN,"Urea Nitrogen, Urine 78 mg/dL, Creatinine, Uri..."
120471,29347,146160,2198-07-07,2198-07-07 00:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,MYOCARDIAL INFARCTION/CATH,V4501,F,...,2198-07-02 02:18:00,51265.0,147,K/uL,abnormal,2142-07-07,20454 days,NaN,NaN,"Urea Nitrogen, Urine 78 mg/dL, Creatinine, Uri..."
120472,29347,146160,2198-07-07,2198-07-07 00:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,MYOCARDIAL INFARCTION/CATH,41011,F,...,2198-07-02 02:18:00,51279.0,3.87,m/uL,abnormal,2142-07-07,20454 days,NaN,NaN,"Urea Nitrogen, Urine 78 mg/dL, Creatinine, Uri..."


***Run the following cell only if you have reduced the dataset size previously, otherwise it will crash your memory***

In [ ]:
# This step can be computationally intensive, consider optimizing by pre-filtering noteevents
noteevents_filtered = noteevents_selected[['SUBJECT_ID', 'HADM_ID', 'TEXT']]
results_df = results_df.merge(noteevents_filtered, on=['SUBJECT_ID', 'HADM_ID'], how='left')
results_df

In [5]:
results_df.to_csv("MIMIC without diag.csv.csv")

## Old Symptoms Extraction from NoteEvents

#### This method extracts symptoms from patient's clinical notes based on some regex patterns

In [2]:
import pandas as pd
import spacy
from spacy.matcher import Matcher
from tqdm import tqdm

# Load the English NLP model
nlp = spacy.load('en_core_web_sm')

# Initialize the Matcher with the shared vocabulary
matcher = Matcher(nlp.vocab)

# Define more specific patterns to capture common ways symptoms might be described in clinical notes
patterns = [
    [{"LOWER": "complaints"}, {"LOWER": "of"}],  # to match "complaints of..."
    [{"LOWER": "complaint"}, {"IS_SPACE": True, "OP": "*"}, {"LOWER": "is"}],  # "Chief complaint is..."
    [{"LOWER": "presenting"}, {"LOWER": "symptoms"}],  # "presenting symptoms are..."
    [{"LOWER": "presents"}, {"LOWER": "with"}],  # "patient presents with..."
    [{"LOWER": "symptoms"}, {"LOWER": "include"}],  # "symptoms include..."
    [{"LOWER": "symptom"}, {"LOWER": "of"}]  # For any singular references to symptoms
]

# Add the pattern to the matcher
matcher.add("SYMPTOM_DETECTION", patterns)

# Function to apply the matcher to the text and extract a reasonable amount of following content
def extract_symptoms(text):
    doc = nlp(text)
    matches = matcher(doc)
    symptoms = []
    for match_id, start, end in matches:
        extended_end = end + 10 if end + 10 < len(doc) else len(doc)
        span = doc[start:extended_end]
        symptoms.append(span.text)
    return symptoms

# Use tqdm to show a progress bar when applying the function across the DataFrame
tqdm.pandas(desc="Extracting symptoms")
noteevents_selected['extracted_symptoms'] = noteevents_selected['TEXT'].progress_apply(extract_symptoms)

# Save or display the updated DataFrame
noteevents_selected.to_csv('updated_noteevents.csv', index=False)
noteevents_selected[['TEXT', 'extracted_symptoms']].head()

Extracting symptoms: 100%|██████████| 25324/25324 [31:30<00:00, 13.39it/s]  
/tmp/ipykernel_33/3140911536.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noteevents_selected['extracted_symptoms'] = noteevents_selected['TEXT'].progress_apply(extract_symptoms)


,TEXT,extracted_symptoms
54,Admission Date: [**2193-5-30**] Dischar...,[]
55,Admission Date: [**2193-6-9**] Discharg...,[]
108,Admission Date: [**2196-10-14**] ...,[]
187,Admission Date: [**2166-3-17**] ...,[]
188,Admission Date: [**2168-5-6**] D...,[]


## New Symptoms Extraction from NoteEvents

#### This method extracts symptoms from patient's clinical notes based on a set of symptoms given to be extracted if they are found

In [4]:
import spacy
from spacy.matcher import Matcher
from tqdm import tqdm

# Load spaCy model
nlp = spacy.load('en_core_web_sm')

# Step 2: Initialize Matcher for pattern-based symptom extraction
matcher = Matcher(nlp.vocab)
patterns = [
    [{"LOWER": "complaints"}, {"LOWER": "of"}, {"OP": "*"}],  # Capture "complaints of [symptoms]"
    [{"LOWER": "complaint"}, {"IS_SPACE": True, "OP": "*"}, {"LOWER": "is"}, {"OP": "*"}],  # "Chief complaint is..."
    [{"LOWER": "presenting"}, {"LOWER": "symptoms"}, {"OP": "*"}],  # "presenting symptoms are..."
    [{"LOWER": "presents"}, {"LOWER": "with"}, {"OP": "*"}],  # "presents with [symptoms]"
    [{"LOWER": "symptoms"}, {"LOWER": "include"}, {"OP": "*"}],  # "symptoms include [symptoms]"
    [{"LOWER": "symptom"}, {"LOWER": "of"}, {"OP": "*"}]  # For any singular references to symptoms
]
matcher.add("SYMPTOM_DETECTION", patterns)

# Step 3: Define list of symptoms for keyword matching
all_symptoms = ['heart murmur', 'hardening', 'neck pain', 'spasm','ulcers on tongue',
                'orthopnea', 'stomatitis', 'abscess', 'pus filled pimples', 'deaf',
                'walking difficulty', 'ileitis','acute gastritis', 'hematoma',
                'mal de mer', 'skin rash','inclusion body myositis', "Kernig's sign",
                'radiation sickness','wasting', 'scratching', 'laryngopharyngitis',
                'lumbago', 'squint','parotitis', 'smell disturbance', 'balanitis',
                'exanthema',
       'oedema', 'mucoid sputum', 'drying and tingling lips', 'uraturia',
       'growing pains', 'breathing difficulty', 'hyperlipoidemia',
       'familial hypercholesterolemia', 'haemothorax', 'hematocele',
       'acute glossitis', 'fasciculation', 'acute liver failure',
       'trismus', 'stomach rumbles', 'minimal brain dysfunction',
       'kraurosis', 'esophagitis', 'griping', 'ophthalmia neonatorum',
       'swelling of stomach', 'epidemic encephalitis', 'sweating',
       'stomach upset', 'spotting  urination', 'birth pangs', 'wrick',
       'polyuria', 'chafe', "Paget's disease", 'pulmonary congestion',
       'primary dysmenorrhea', 'oophoritis', 'hydrarthrosis',
       'Marie-Strumpell disease', 'hearing loss', 'toxic look (typhos)',
       'receiving blood transfusion', 'oesophagitis', 'lymphangitis',
       'watering from eyes', 'skin peeling', 'carditis', 'dacryocystitis',
       'keloid', 'phantom limb syndrome', 'regional ileitis',
       'stomachache', 'bagascosis', 'hypersplenism',
       'malabsorption syndrome', 'photophobia', 'sweaty', 'ADD',
       'movement impairment', 'haemoptysis', 'formication', 'agony',
       'lead colic', 'myositis trichinosa', 'myodynia', 'migraine',
       'memory issue', 'haematocyturia', 'encephalitis', 'neuralgy',
       'hip joint pain', 'heartburn', 'ophthalmitis', 'hyperlipaemia',
       'coma', 'lipemia', 'swelled lymph nodes', 'dischromic  patches',
       'pain', 'cervical root syndrome', 'nauseated', 'megrim',
       'glossodynia exfoliativa', 'afterpains', 'hypocalcemia',
       'prominent veins on calf', 'effect', 'abnormal menstruation',
       "Reiter's disease", 'cough', 'rusty sputum',
       'inflammatory bowel disease', 'ammoniuria', 'pleuralgia',
       'yellow crust ooze', 'tumidity', 'encephalomyelitis',
       'ulcerative colitis', "Noonan's syndrome", 'swelling', 'thirsty',
       'rheumatoid spondylitis', 'lethargy', 'tooth pain', 'colitis',
       'panencephalitis', 'looseness', 'looseness of the bowels',
       'acetonemia', 'yellowing of eyes', 'back pain', 'cephalitis',
       'cerebral edema', 'jaundice of the newborn', 'otitis interna',
       'swelling joints', 'ague', 'nephrosis', 'urtication',
       'Williams syndrome', 'proctitis', 'anaemia', 'mastalgia',
       'steatorrhea', 'weak', 'autism', 'ache', 'proctalgia',
       'epididymitis', 'chorditis', 'iridoncus', 'blackheads',
       'otitis media', 'hemoglobinuria', 'tympanitis',
       'lateral humeral epicondylitis', 'scleredema',
       'temporal arteritis', 'urodynia', 'skin eruption', 'ear pain',
       'Van Bogaert encephalitis', 'hyperlipidaemia',
       'periodic apnea of the newborn', 'smarting', 'prodrome',
       'uricaciduria', 'yellowish skin', 'shin splints', 'laminitis',
       'red spots over body', 'hypocalcaemia', 'ochronosis',
       'swollen legs', "writer's cramp", 'haemoglobinuria',
       'loss of balance', 'pain behind the eyes', 'sleepy',
       'Persian Gulf illness', 'tendinitis', 'cystoid macular edema',
       'hydrophobia', 'kaliuresis', 'speech issue', 'intumescency',
       'small dents in nails', 'SOB', 'backache', 'thyroiditis',
       'eosinopenia', 'attention deficit hyperactivity disorder',
       'inclusion body encephalitis', 'infectious polyneuritis',
       'tracheobronchitis', "Kaposi's varicelliform eruption", 'callus',
       'photalgia', 'vasculitis', 'areflexia', 'keratitis', 'mood swings',
       'enanthem', 'rubor', 'kraurosis vulvae', 'tinnitus',
       'abscessed tooth', 'febricity', 'alveolitis', 'chloasma',
       'peritonitis', 'anasarca', 'alkalinuria', "Crohn's disease",
       'nodal skin eruptions', 'glossodynia', 'multiple neuritis',
       'podalgia', 'polymyositis', 'ketoaciduria', 'abdominal pain',
       "Montezuma's revenge", 'blister', 'graphospasm', 'tenonitis',
       'aching', 'muscle spasm', 'fever', 'knee pain',
       'chronic gastritis', 'leg pain', 'vaccination',
       'West Nile encephalitis', 'air sickness', 'pollinosis',
       'palpitation', 'pockmark', 'tonsillitis', 'kinetosis', 'palsy',
       'MBD', 'angiitis', 'scalenus syndrome', 'wheeziness', 'phrenitis',
       'hypercalcemia', 'chronic glossitis', 'vomiting', 'megacardia',
       'polyneuritis', 'toxic shock', 'lipaemia', 'hypercholesteremia',
       'excessive hunger', 'head pain', 'rectum pain', 'cholecystitis',
       'alkaluria', 'thoracic outlet syndrome', 'endocarditis',
       'apyretic tetanus', 'ventricular fibrillation', 'hyperlipidemia',
       "Quincke's edema", 'eruption', 'hypercalciuria',
       'radiation syndrome', 'FAS', 'hydrops', 'patches in throat',
       'melasma', 'stomach bleeding', 'myometritis', 'hyperkalemia',
       'scleritis', "Graves' disease", 'inflammatory nails',
       'fetal alcohol syndrome', 'odynophagia', 'lipoidaemia',
       'yellow urine', 'pancreatitis', 'regional enteritis', 'itching',
       'shortness of breath', 'oscheocoele', 'burning', 'blepharism',
       'vesiculitis', 'tendonitis', 'episcleritis', 'tetanilla',
       'keratomalacia', "Fallot's syndrome", 'cardiomegaly', 'hemoptysis',
       'aftereffect', 'headache', "Fallot's tetralogy", 'anemia',
       'squeamishness', 'black tongue', 'febrility', 'cellulitis',
       'retrobulbar neuritis', 'spastic colon', 'myoglobinuria',
       'subacute inclusion body encephalitis', 'dyspnoea',
       'breathlessness', 'swollen extremeties', "Reiter's syndrome",
       'tic douloureux', 'prodroma', 'cardiac murmur', 'cicatrice',
       'pericarditis', 'lipidemia', 'myoclonus', 'secondary dysmenorrhea',
       'obesity', 'constipation', 'irritable bowel syndrome',
       'restless legs syndrome', 'prostatitis', 'salpingitis',
       'defecation issue', 'hypercholesterolemia',
       'maple syrup urine disease', 'diuresis', 'purulence', 'stinging',
       'twitching', 'Gilles de la Tourette syndrome', 'chest pain',
       'dehydration', 'congestion', 'arthralgia', 'lipoidemia',
       'burning micturition', 'the shits', 'pyrexia', 'unsteadiness',
       'subacute sclerosing panencephalitis', 'phalangitis', 'sneezing',
       "Munchausen's syndrome", 'crib death', 'giddiness', 'uratemia',
       'hyperemia', 'pang', 'sialadenitis', 'stomach pain', 'hiccup',
       'cheloid', 'smartness', 'hairy tongue', 'runny nose', 'vaginitis',
       'hydrothorax', 'peritoneal inflammation', 'periodic edema',
       'furry tongue', 'muscle weakness', "Klinefelter's syndrome",
       'snotty nose', 'tarsitis', 'roseola', 'bloody stool', 'rachitis',
       'trichiniasis', 'urticaria', 'posthitis', 'pelvis pain',
       'writing difficulty', 'cot death', 'weight loss',
       'history of alcohol consumption', 'Ramsay Hunt syndrome',
       'vasovesiculitis', 'haematuria', 'gastralgia', 'lymphedema',
       'balanoposthitis', 'epiglottitis', 'mucous colitis', 'throb',
       'haemorrhoid', "housemaid's knee", 'jejunoileitis', 'lump',
       'nausea', 'hay fever', 'suffering', 'otitis', 'pancarditis',
       'tetany', 'dark urine', 'keratoiritis', "Landry's paralysis",
       'tenosynovitis', 'physiological jaundice of the newborn',
       'anxiety', 'aortitis', 'hyperkinetic syndrome', 'appendicitis',
       'cervicitis', 'altered sensorium', "Takayasu's arteritis",
       'barking cough', 'periarteritis', 'cramp', 'hyperglycemia',
       'silver like dusting', 'aerodontalgia', 'iridocyclitis',
       'kernicterus', 'osteomyelitis', 'menorrhagia',
       'pain during bowel movements', 'rebound tenderness',
       'pansinusitis', 'pain in anal region', 'nephrotic syndrome',
       'cluster headache', 'metritis', 'tennis elbow', 'burn',
       'equine encephalomyelitis', 'phlebitis', 'intumescence',
       'increased appetite', 'hot flash', 'hoarse', 'acrocyanosis',
       'gut pain', 'infant death', 'dry mouth', 'crepitation rale',
       'histamine headache', 'retinitis', 'diarrhoea', "Conn's syndrome",
       'oscheocele', 'subacute sclerosing leukoencephalitis',
       'swallowing difficulty', 'snuffles', 'costalgia',
       'blurred and distorted vision', 'hives', 'acute encephalitis',
       'haematocoele', 'muscae volitantes', 'crick', 'enanthema',
       'icterus neonatorum', 'thermalgesia', 'weak heart', 'joint pain',
       'atrophy', 'fecal impaction', 'continuous feel of urine',
       'restlessness', 'stomach ache', 'dyspepsia', 'dizzy',
       'secondary amenorrhea', 'sleeping sickness', 'hemoglobinemia',
       'leukoencephalitis', 'polyarteritis', "Bosin's disease",
       'arteritis', 'skin pain', 'funiculitis', 'hyperaemia',
       'brain edema', 'proteinuria', 'taste disturbance',
       'laryngotracheobronchitis', 'parametritis', "Moeller's glossitis",
       'albuminuria', 'bellyache', 'fibromyositis', 'hypoglycaemia',
       'labour pains', 'pinkeye', 'conjunctivitis', 'hyperpyrexia',
       'sore throat', 'enlarged thyroid', 'branched chain ketoaciduria',
       'haematocele', 'blood in sputum', 'irritation in anus', 'flush',
       'withdrawal symptom', 'coryza', 'bubo', 'equine encephalitis',
       'cheilitis', 'giant hives', 'vertigo', 'fast heart rate',
       'hurting', 'throe', 'acidity', 'distention of abdomen', 'furring',
       'puffy face and eyes', 'myalgia', 'palpitations',
       'premenstrual syndrome', 'cicatrix', 'haemoglobinemia',
       'calcification', 'costochondritis', 'ulitis', 'infantile autism',
       'passage of gases', 'lateral epicondylitis', 'seasickness',
       'vulvovaginitis', 'cold hands and feets', 'visual disturbances',
       "farmer's lung", 'bruising', 'nephralgia', 'Ekbom syndrome',
       'hypoglycemia', 'systolic murmur', 'sternutation', 'callosity',
       'inflammation', 'folliculitis', 'rhinitis', 'fructosuria',
       'the trots', "Reye's syndrome", 'upset stomach',
       'Gulf War syndrome', 'irritability', 'sting', 'ulalgia',
       'pleurodynia', 'neuritis', 'sciatica', 'clubbing', 'monocytosis',
       'Klinefelter syndrome', 'red sore around nose', 'hyperglycaemia',
       'efflorescence', 'tension headache', 'eczema vaccinatum',
       'herpes encephalitis', 'causalgia', 'jejunitis',
       'primary amenorrhea', 'abdomen pain', 'encephalomeningitis',
       'fluid overload', 'encephalitis lethargica', 'sleep disturbance',
       'hemicrania', 'numbness', 'uveitis', 'XXY-syndrome',
       'swollen blood vessels', 'high fever', 'referred pain',
       'intermittent cramp', 'glossitis', 'glucosuria',
       'exophthalmic goiter', 'tetralogy of Fallot', 'aminoaciduria',
       'musca volitans', 'toxic shock syndrome', 'trigeminal neuralgia',
       'amyotrophy', 'weight gain', 'mask of pregnancy',
       'ankylosing spondylitis', 'trichinosis', 'apnea',
       'redness of eyes', 'exophthalmos', 'puffiness',
       'irregular sugar level', 'jaundice', 'ketonuria', 'hemothorax',
       'necrotizing enteritis', 'pneumonitis', 'continuous sneezing',
       'attention deficit disorder', 'fibrillation', 'intestinal colic',
       'murmur', 'blepharitis', 'labyrinthitis', 'vulvitis',
       'hypernatremia', 'gastritis', 'thrombocytosis',
       "Tourette's syndrome", 'foul smell of urine',
       'weakness of one body side', 'movement stiffness',
       'intermittent tetanus', 'otitis externa', 'splenitis',
       'odontalgia', 'qualm', 'nasal congestion', 'hiccough',
       'car sickness', 'rubella panencephalitis', 'meningoencephalitis',
       'chafing', 'tenesmus', 'slurred speech', 'naupathia',
       'tendosynovitis', 'cramps', 'dropsy', 'iritis', 'tumidness',
       'charley horse', 'light-headed', "thresher's lung", 'rhinorrhea',
       'head ache', 'lymphogranuloma', 'extra marital contacts',
       'belly pain', 'muscle wasting', 'bagassosis', 'cerebromeningitis',
       'hemosiderosis', 'mastoiditis', 'radiculitis', 'spermatocele',
       'bursitis', 'earache', 'TSS', 'uvulitis', 'blepharospasm',
       'lymphuria', 'mild fever', 'edema', 'internal itching',
       'dyschezia', 'chemosis', 'malaise', 'megalocardia', 'meralgia',
       'spinning movements', 'myelitis', 'irregularity', 'colpocystitis',
       "Horner's syndrome", 'paresthesia', 'opisthotonos',
       'lethargic encephalitis', 'endometritis', 'natriuresis',
       'amenorrhea', "Dawson's encephalitis", 'sunken eyes', 'toothache',
       'rheumatic aortitis', 'dermatomyositis', 'hyponatremia',
       'minimal brain damage', 'haematoma', 'bloat',
       'lack of concentration', 'corditis', 'fatigue', 'dizziness',
       'dysmenorrhea', "Koplik's spots", 'neuralgia',
       'atrial fibrillation', 'cyanosis', 'chronic pain', 'hemorrhoid',
       'sleep apnea', 'tendonous synovitis', 'chorioretinitis',
       'ureteritis', 'osteitis deformans', 'engorgement', 'hyperlipemia',
       'weakness in limbs', 'chiralgia', 'diverticulitis',
       'subacute bacterial endocarditis', 'melagra', 'iridokeratitis',
       'clavus', 'loss of appetite', "Tietze's syndrome", 'valvulitis',
       'lightheadedness', 'shivering', 'purulency', 'fart', 'tired',
       'sclerosing leukoencephalitis', 'metralgia', 'quartan', 'pyuria',
       'glossalgia', 'distress', 'lazy eye', 'ketosis', 'stridor',
       'papilledema', 'angioedema', 'cephalalgia', 'hypoproteinemia',
       'keratoconjunctivitis', 'myocardial inflammation', 'ophthalmia',
       'bummer', 'adenitis', 'brittle nails', 'belch', 'otalgia',
       'Munchausen syndrome', 'stuffiness', 'hypermenorrhea', 'kaluresis',
       'chesty cough', 'SSPE', 'chill', 'pulseless disease',
       'throat irritation', 'cervical disc syndrome', 'mastitis',
       'phlegm', 'enlarged heart', "Jacquemier's sign", 'loss of smell',
       'Waterhouse-Friderichsen syndrome', 'oliguria', 'stiff neck',
       'ADHD', 'bunion', 'hematocyturia', 'muscle pain', 'torment',
       'spondylitis', 'painful walking', 'dry socket', 'morning sickness',
       'exanthem', 'orchitis', 'soreness', 'airsickness',
       'haemosiderosis', 'pyrosis', 'chills and fever', 'motion sickness',
       'ketonemia', 'sinus pressure', 'hematuria', "painter's colic",
       'eosinophilia', 'myocarditis', 'fibrositis', 'waterworks',
       'endocervicitis', 'amyotrophia', 'chills', 'ovaritis',
       'indigestion', 'dyspnea', 'keratoscleritis', 'hypokalemia',
       'renal colic', 'sneeze', 'lymphadenitis', 'orchidalgia',
       'sudden infant death syndrome', 'hypercalcaemia', 'cholangitis',
       'hypercalcinuria', 'urination issue', 'colpitis', 'vellication',
       'paraesthesia', 'labor pains', 'postnasal drip', 'queasiness',
       'depression', 'sick headache', 'endarteritis', 'tabes',
       'acetonuria', 'spots', 'catarrh', 'tracheitis', 'laryngitis',
       'tic', 'Zollinger-Ellison syndrome', 'osteitis',
       'herpes simplex encephalitis', 'intertrigo', 'epicondylitis',
       'icterus', 'family history', 'lipidaemia', 'coughing',
       'festination', 'Kayser-Fleischer ring', 'keratalgia', 'PMS',
       'aura', 'vesicular stomatitis', 'obstipation', 'prickly heat',
       'allergic rhinitis', 'meningism', 'water on the knee', 'scurring',
       'twitch', 'tumescence', 'hyperlipoidaemia', 'myositis',
       'sinusitis', 'heat rash', 'miliaria', 'glycosuria', 'labor pain',
       'diarrhea', 'feverishness', 'enteritis', 'bladder discomfort',
       'gripes', 'gastric problems', 'nettle rash', 'floater',
       'vision issue', 'hematocoele', 'colic', 'lumbar pain',
       'thrombophlebitis', 'tenderness', 'excruciation', 'torture',
       'synovitis', 'sinus headache', 'wasting away', 'sword-cut',
       'Chinese restaurant syndrome', 'receiving unsterile injections']
unique_symptoms = [symptom.strip().lower() for symptom in all_symptoms]

# Step 4: Function to handle negations and extract symptoms (both single and multi-word phrases)
def is_negated(token):
    """Check if the symptom is negated by looking for a negation modifier."""
    for child in token.children:
        if child.dep_ == 'neg':
            return True
    return False

def extract_symptoms(text):
    doc = nlp(text)

    symptoms_found = []

    # 1. Pattern-based matching for sentence-like symptoms
    matches = matcher(doc)
    for match_id, start, end in matches:
        # Extend the span after the match to capture the symptom phrase, skipping the starters
        if doc[start:end].text.startswith(('presents', 'complaints', 'symptoms', 'complaint')):
            symptom_start = end  # Start capturing symptoms after the pattern
            symptom_span = doc[symptom_start:symptom_start + 10] if symptom_start + 10 < len(doc) else doc[symptom_start:]
            symptoms_found.append(symptom_span.text.strip())

    # 2. Keyword-based symptom extraction for single symptoms
    for token in doc:
        if token.text.lower() in unique_symptoms and not is_negated(token):
            symptoms_found.append(token.text)

    return list(set(symptoms_found))  # Remove duplicates

# Step 5: Apply the function to the note events and extract symptoms
tqdm.pandas(desc="Extracting symptoms")
noteevents_selected['extracted_symptoms'] = noteevents_selected['TEXT'].progress_apply(extract_symptoms)

# Step 6: Save the extracted symptoms
noteevents_selected.to_csv('updated_noteevents_with_combined_symptoms.csv', index=False)

# Display the extracted symptoms
print(noteevents_selected[['TEXT', 'extracted_symptoms']].head())

Extracting symptoms: 100%|██████████| 3869/3869 [05:36<00:00, 11.50it/s] 
/tmp/ipykernel_33/1941458291.py:295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noteevents_selected['extracted_symptoms'] = noteevents_selected['TEXT'].progress_apply(extract_symptoms)


                                                   TEXT  \
54    Admission Date: [**2193-5-30**]        Dischar...   
55    Admission Date: [**2193-6-9**]        Discharg...   
259   Admission Date:  [**2110-1-6**]       Discharg...   
1548  Admission Date:  [**2124-3-1**]              D...   
2484  Admission Date:  [**2131-6-27**]              ...   

                                     extracted_symptoms  
54        [cyanosis, edema, pain, clubbing, Depression]  
55       [distress, swelling, edema, Depression, apnea]  
259   [one to two q.four hours p.r.n.\n10., mg b.i.d...  
1548  [] 1:52 pm URINE      Source: Catheter., , Com...  
2484  [be performed in the patient as his\ncode stat...  


In [ ]:
# Returning only the rows which they have symptoms to visualize the extracted symptoms 
filtered_noteevents = noteevents_selected[noteevents_selected['extracted_symptoms'].apply(lambda x: len(x) > 0)]
filtered_noteevents

In [6]:
results_df

,SUBJECT_ID,HADM_ID,STARTDATE,ENDDATE,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,DIAGNOSIS,ICD9_CODE,GENDER,...,LAB_CHARTTIME,LAB_ITEMID,LAB_VALUE,LAB_VALUEUOM,LAB_FLAG,nearest_chartdate,date_diff,CHARTDATE,extracted_symptoms,LABTESTS
0,1291,141087,2100-07-15,2100-07-23 00:00:00,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,NEWBORN,V053,F,...,NaN,NaN,NaN,NaN,NaN,2106-09-16,2254 days,NaN,NaN,"Bilirubin, Total 12.3 mg/dL, Bilirubin, Direct..."
1,1291,141087,2100-07-15,2100-07-23 00:00:00,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,NEWBORN,V290,F,...,NaN,NaN,NaN,NaN,NaN,2106-09-16,2254 days,NaN,NaN,"Bilirubin, Total 12.3 mg/dL, Bilirubin, Direct..."
2,1291,141087,2100-07-15,2100-07-23 00:00:00,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,NEWBORN,7630,F,...,NaN,NaN,NaN,NaN,NaN,2106-09-16,2254 days,NaN,NaN,"Bilirubin, Total 12.3 mg/dL, Bilirubin, Direct..."
3,1291,141087,2100-07-15,2100-07-23 00:00:00,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,NEWBORN,76527,F,...,2100-07-04 09:52:00,51277.0,16.2,%,abnormal,2106-09-16,2254 days,NaN,NaN,"Bilirubin, Total 12.3 mg/dL, Bilirubin, Direct..."
4,1291,141087,2100-07-15,2100-07-23 00:00:00,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,NEWBORN,7793,F,...,2100-07-04 09:52:00,51256.0,42,%,abnormal,2106-09-16,2254 days,NaN,NaN,"Bilirubin, Total 12.3 mg/dL, Bilirubin, Direct..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120469,29347,146160,2198-07-07,2198-07-07 00:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,MYOCARDIAL INFARCTION/CATH,41011,F,...,2198-07-03 05:12:00,51237.0,2.9,NaN,abnormal,2142-07-07,20454 days,NaN,NaN,"Urea Nitrogen, Urine 78 mg/dL, Creatinine, Uri..."
120470,29347,146160,2198-07-07,2198-07-07 00:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,MYOCARDIAL INFARCTION/CATH,V4364,F,...,2198-07-02 02:18:00,51274.0,33.0,sec,abnormal,2142-07-07,20454 days,NaN,NaN,"Urea Nitrogen, Urine 78 mg/dL, Creatinine, Uri..."
120471,29347,146160,2198-07-07,2198-07-07 00:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,MYOCARDIAL INFARCTION/CATH,V4501,F,...,2198-07-02 02:18:00,51265.0,147,K/uL,abnormal,2142-07-07,20454 days,NaN,NaN,"Urea Nitrogen, Urine 78 mg/dL, Creatinine, Uri..."
120472,29347,146160,2198-07-07,2198-07-07 00:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,MYOCARDIAL INFARCTION/CATH,41011,F,...,2198-07-02 02:18:00,51279.0,3.87,m/uL,abnormal,2142-07-07,20454 days,NaN,NaN,"Urea Nitrogen, Urine 78 mg/dL, Creatinine, Uri..."


In [8]:
# Drop rows with missing STARTDATE values
results_df = results_df.dropna(subset=['STARTDATE'])

# Drop rows with missing CHARTDATE values in filtered_noteevents
filtered_noteevents = filtered_noteevents.dropna(subset=['CHARTDATE'])

results_df['STARTDATE'] = pd.to_datetime(results_df['STARTDATE'])
filtered_noteevents['CHARTDATE'] = pd.to_datetime(filtered_noteevents['CHARTDATE'])

# Example to display the minimum difference in dates for a subset
results_df['nearest_chartdate'] = pd.merge_asof(
    results_df.sort_values('STARTDATE'),
    filtered_noteevents[['CHARTDATE']].sort_values('CHARTDATE'),
    left_on='STARTDATE',
    right_on='CHARTDATE',
    direction='nearest',
    allow_exact_matches=True
)['CHARTDATE']

# Calculate the time difference
results_df['date_diff'] = (results_df['STARTDATE'] - results_df['nearest_chartdate']).abs()

# Check statistics of date differences
print(results_df['date_diff'].describe())

KeyError: 'CHARTDATE'

In [11]:
filtered_noteevents = filtered_noteevents.dropna(subset=['HADM_ID'])
filtered_noteevents['HADM_ID'] = filtered_noteevents['HADM_ID'].astype('int64')


# Ensure that SUBJECT_ID and HADM_ID have the same data type in both DataFrames
results_df['SUBJECT_ID'] = results_df['SUBJECT_ID'].astype('int64')
results_df['HADM_ID'] = results_df['HADM_ID'].astype('int64')

filtered_noteevents['SUBJECT_ID'] = filtered_noteevents['SUBJECT_ID'].astype('int64')
filtered_noteevents['HADM_ID'] = filtered_noteevents['HADM_ID'].astype('int64')

# Adjust tolerance to limit the maximum time difference
import pandas as pd

merged_df = pd.merge_asof(
    results_df.sort_values('STARTDATE'),
    filtered_noteevents[['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'extracted_symptoms']].sort_values('CHARTDATE'),
    left_on='STARTDATE',
    right_on='CHARTDATE',
    by=['SUBJECT_ID', 'HADM_ID'],
    direction='nearest',
    tolerance=pd.Timedelta(days=7)  # Only consider matches within 7 days
)

print("Non-null symptoms count:", merged_df['extracted_symptoms'].notna().sum())


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [9]:
merged_df.isna().sum()

NameError: name 'merged_df' is not defined

In [9]:
merged_df[merged_df['extracted_symptoms']== np.nan]

,SUBJECT_ID,HADM_ID,STARTDATE,ENDDATE,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,DIAGNOSIS,ICD9_CODE,GENDER,...,ROUTE,LAB_CHARTTIME,LAB_ITEMID,LAB_VALUE,LAB_VALUEUOM,LAB_FLAG,nearest_chartdate,date_diff,CHARTDATE,extracted_symptoms


In [10]:
import pandas as pd

# Ensure correct data types and copy dataframes to avoid SettingWithCopyWarning
results_df = results_df.copy()
filtered_noteevents = filtered_noteevents.copy()

results_df['SUBJECT_ID'] = results_df['SUBJECT_ID'].astype(int)
results_df['HADM_ID'] = results_df['HADM_ID'].astype(int)
results_df['STARTDATE'] = pd.to_datetime(results_df['STARTDATE'])

filtered_noteevents['SUBJECT_ID'] = filtered_noteevents['SUBJECT_ID'].astype(int)
filtered_noteevents['HADM_ID'] = filtered_noteevents['HADM_ID'].astype(int)
filtered_noteevents['CHARTDATE'] = pd.to_datetime(filtered_noteevents['CHARTDATE'])

# Sort DataFrames by the datetime columns for merge_asof
results_df = results_df.sort_values('STARTDATE')
filtered_noteevents = filtered_noteevents.sort_values('CHARTDATE')

from tqdm.auto import tqdm  # Using auto to ensure compatibility with notebooks and terminals

# Inform tqdm of the operation about to be performed
tqdm.pandas(desc="Preparing for merge")

# Perform the merge_asof
merged_df = pd.merge_asof(
    results_df,
    filtered_noteevents[['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'extracted_symptoms']],
    left_on='STARTDATE',
    right_on='CHARTDATE',
    by=['SUBJECT_ID', 'HADM_ID'],
    direction='nearest'
)

print("Merge completed.")
print(merged_df.head())

merged_df.to_csv('merged_results.csv', index=False)

Merge completed.
   SUBJECT_ID  HADM_ID  STARTDATE              ENDDATE ADMISSION_TYPE  \
0        1291   141087 2100-07-15  2100-07-23 00:00:00        NEWBORN   
1        1291   141087 2100-07-15  2100-07-23 00:00:00        NEWBORN   
2        1291   141087 2100-07-15  2100-07-23 00:00:00        NEWBORN   
3        1291   141087 2100-07-15  2100-07-23 00:00:00        NEWBORN   
4        1291   141087 2100-07-15  2100-07-23 00:00:00        NEWBORN   

          ADMISSION_LOCATION DISCHARGE_LOCATION DIAGNOSIS ICD9_CODE GENDER  \
0  PHYS REFERRAL/NORMAL DELI               HOME   NEWBORN      V053      F   
1  PHYS REFERRAL/NORMAL DELI               HOME   NEWBORN      V290      F   
2  PHYS REFERRAL/NORMAL DELI               HOME   NEWBORN      7630      F   
3  PHYS REFERRAL/NORMAL DELI               HOME   NEWBORN     76527      F   
4  PHYS REFERRAL/NORMAL DELI               HOME   NEWBORN      7793      F   

   ...  ROUTE        LAB_CHARTTIME LAB_ITEMID LAB_VALUE LAB_VALUEUOM  \
0  

In [11]:
non_null_symptoms_df = merged_df[merged_df['extracted_symptoms'].notna()]
non_null_symptoms_df

,SUBJECT_ID,HADM_ID,STARTDATE,ENDDATE,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,DIAGNOSIS,ICD9_CODE,GENDER,...,ROUTE,LAB_CHARTTIME,LAB_ITEMID,LAB_VALUE,LAB_VALUEUOM,LAB_FLAG,nearest_chartdate,date_diff,CHARTDATE,extracted_symptoms
24,79838,134899,2100-08-19,2100-08-19 00:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,VFIB ARREST,490,M,...,IV DRIP,NaN,NaN,NaN,NaN,NaN,2188-04-01,32002 days,2100-08-21,[presents with neurological\n deficits on phy...
25,79838,134899,2100-08-19,2100-08-19 00:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,VFIB ARREST,04119,M,...,PO,NaN,NaN,NaN,NaN,NaN,2188-04-01,32002 days,2100-08-21,[presents with neurological\n deficits on phy...
26,79838,134899,2100-08-19,2100-08-19 00:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,VFIB ARREST,42821,M,...,IV DRIP,NaN,NaN,NaN,NaN,NaN,2188-04-01,32002 days,2100-08-21,[presents with neurological\n deficits on phy...
27,79838,134899,2100-08-19,2100-08-19 00:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,VFIB ARREST,V4582,M,...,PO,NaN,NaN,NaN,NaN,NaN,2188-04-01,32002 days,2100-08-21,[presents with neurological\n deficits on phy...
28,79838,134899,2100-08-19,2100-08-19 00:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,VFIB ARREST,51881,M,...,IV DRIP,NaN,NaN,NaN,NaN,NaN,2188-04-01,32002 days,2100-08-21,[presents with neurological\n deficits on phy...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827001,98089,179750,2201-02-24,2201-02-25 00:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOSPICE-MEDICAL FACILITY,UPPER GASTROINTESTINAL BLEED,56089,M,...,PO/NG,NaN,NaN,NaN,NaN,NaN,2200-01-28,392 days,2201-02-16,[presents with obstruction seen on colonoscopy...
827002,98089,179750,2201-02-24,2201-02-25 00:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOSPICE-MEDICAL FACILITY,UPPER GASTROINTESTINAL BLEED,1976,M,...,PO/NG,NaN,NaN,NaN,NaN,NaN,2200-01-28,392 days,2201-02-16,[presents with obstruction seen on colonoscopy...
827003,98089,179750,2201-02-24,2201-02-25 00:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOSPICE-MEDICAL FACILITY,UPPER GASTROINTESTINAL BLEED,7821,M,...,PO/NG,NaN,NaN,NaN,NaN,NaN,2200-01-28,392 days,2201-02-16,[presents with obstruction seen on colonoscopy...
827004,98089,179750,2201-02-24,2201-02-25 00:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOSPICE-MEDICAL FACILITY,UPPER GASTROINTESTINAL BLEED,40390,M,...,PO/NG,NaN,NaN,NaN,NaN,NaN,2200-01-28,392 days,2201-02-16,[presents with obstruction seen on colonoscopy...


In [12]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 827006 entries, 0 to 827005
Data columns (total 28 columns):
 #   Column              Non-Null Count   Dtype          
---  ------              --------------   -----          
 0   SUBJECT_ID          827006 non-null  int64          
 1   HADM_ID             827006 non-null  int64          
 2   STARTDATE           827006 non-null  datetime64[ns] 
 3   ENDDATE             826283 non-null  object         
 4   ADMISSION_TYPE      827006 non-null  object         
 5   ADMISSION_LOCATION  827006 non-null  object         
 6   DISCHARGE_LOCATION  827006 non-null  object         
 7   DIAGNOSIS           827006 non-null  object         
 8   ICD9_CODE           827006 non-null  object         
 9   GENDER              827006 non-null  object         
 10  AGE                 827006 non-null  int64          
 11  DRUG                827006 non-null  object         
 12  DRUG_TYPE           827006 non-null  object         
 13  PROD_STRENGTH 

In [13]:
merged_df.to_csv('MIMIC without diag 15-11.csv', index=False)